In [1]:
import io
import re
import json
import requests
from warcio import ArchiveIterator
import warcio

### Get URL list of segment

In [2]:
from warcio.archiveiterator import ArchiveIterator
# need to download warc file to data dir
with open('../data/CC-MAIN-20180420081400-20180420101400-00000.warc.gz', 'rb') as stream:
    for record in ArchiveIterator(stream):
        if record.rec_type == 'response':
            print(record.rec_headers.get_header('WARC-Target-URI'))

http://00064.dealerseat.com/news-ed-eventi/news-seat
http://00monthly.com/kouchi/goiken/
http://00secretsweety.ozforex.info/pissysexy21-video
http://018-mb-por.wtfdexter.com/teen/
http://01allice.meteo123.net/sexysweetmasha-torrent
http://01tec.de/
http://01tutorials.com/profile/inetryconydot/150/&sort=3
http://03online.com/news/zdravstvuyte_u_menya_problema_s_nosom/2016-8-11-189497
http://05.rospotrebnadzor.ru/320;jsessionid=C4E01A44166EA22C5DDF2692C1924443
http://05.withtube.info/13792.html
http://05.withtube.info/13990.html
http://053.eisfx.org/2015/05/20/nous-batissons/img_2124/
http://0560.ru/blog/post/6016/
http://0710.com/index.php/cms/pub/login.html
http://080.roomadult.net/index.phtml?PUT=a_show&AID=80690&FID=629256&R2=&CHANNEL=
http://0898hnly.com/yglhj/229.html
http://0d0rm.i05b.com/
http://0ejy8.fvgt.top/
http://0gb.us/map32.txt
http://0lik.ru/video-montag/futages/296605-angel-s-kolcami-v-rukah-futazh.html
http://0tl.ru/?part=lada-granta&&
http://1-72.forumgratuit.org/t6909

### Get HTML of specific URL

Get warc_record_offset and length with ```curl -s https://data.commoncrawl.org/crawl-data/CC-MAIN-2021-10/segments/1614178347293.1/warc/CC-MAIN-20210224165708-20210224195708-00579.warc.gz``` or requests.get():

In [3]:
res = requests.get('https://index.commoncrawl.org/CC-MAIN-2021-10-index?url=https://example.com/')
res.text

'com,example)/ 20210224182523 {"url": "https://example.com/", "mime": "warc/revisit", "status": "304", "length": "582", "offset": "8884434", "filename": "crawl-data/CC-MAIN-2021-10/segments/1614178347293.1/crawldiagnostics/CC-MAIN-20210224165708-20210224195708-00162.warc.gz"}\ncom,example)/ 20210224183127 {"url": "http://example.com/", "mime": "text/html", "mime-detected": "text/html", "status": "200", "digest": "JI6OR3QR4CI526JD6TMMNZNV4QPMPQCH", "length": "1224", "offset": "30681991", "filename": "crawl-data/CC-MAIN-2021-10/segments/1614178347293.1/warc/CC-MAIN-20210224165708-20210224195708-00579.warc.gz", "languages": "eng", "encoding": "UTF-8"}\ncom,example)/ 20210224203718 {"url": "http://example.com/", "mime": "text/html", "mime-detected": "text/html", "status": "200", "digest": "JI6OR3QR4CI526JD6TMMNZNV4QPMPQCH", "length": "1219", "offset": "27824154", "filename": "crawl-data/CC-MAIN-2021-10/segments/1614178347321.0/warc/CC-MAIN-20210224194337-20210224224337-00579.warc.gz", "lan

In [4]:
warc_filename = 'crawl-data/CC-MAIN-2021-10/segments/1614178347293.1/warc/CC-MAIN-20210224165708-20210224195708-00579.warc.gz'
warc_record_offset = 30681991
warc_record_length = 1224

response = requests.get(f'https://data.commoncrawl.org/{warc_filename}',
                        headers={'Range': f'bytes={warc_record_offset}-{warc_record_offset + warc_record_length - 1}'})

with io.BytesIO(response.content) as stream:
    for record in warcio.ArchiveIterator(stream):
        html = record.content_stream().read()

In [5]:
re.findall(b'<title>.+?</title>', html)

[b'<title>Example Domain</title>']

In [6]:
html

b'<!doctype html>\n<html>\n<head>\n    <title>Example Domain</title>\n\n    <meta charset="utf-8" />\n    <meta http-equiv="Content-type" content="text/html; charset=utf-8" />\n    <meta name="viewport" content="width=device-width, initial-scale=1" />\n    <style type="text/css">\n    body {\n        background-color: #f0f0f2;\n        margin: 0;\n        padding: 0;\n        font-family: -apple-system, system-ui, BlinkMacSystemFont, "Segoe UI", "Open Sans", "Helvetica Neue", Helvetica, Arial, sans-serif;\n        \n    }\n    div {\n        width: 600px;\n        margin: 5em auto;\n        padding: 2em;\n        background-color: #fdfdff;\n        border-radius: 0.5em;\n        box-shadow: 2px 3px 7px 2px rgba(0,0,0,0.02);\n    }\n    a:link, a:visited {\n        color: #38488f;\n        text-decoration: none;\n    }\n    @media (max-width: 700px) {\n        div {\n            margin: 0 auto;\n            width: auto;\n        }\n    }\n    </style>    \n</head>\n\n<body>\n<div>\n    

## Test extracting text

mostly from: https://skeptric.com/notebooks/WAT%20WET%20WARC%20-%20Common%20Crawl%20Archives.html 

In [9]:
warc_url = 'https://data.commoncrawl.org/crawl-data/CC-MAIN-2020-24/segments/1590347387219.0/warc/CC-MAIN-20200525032636-20200525062636-00381.warc.gz'
wet_url = warc_url.replace('/warc/', '/wet/').replace('warc.gz', 'warc.wet.gz')
wat_url = warc_url.replace('/warc/', '/wat/').replace('warc.gz', 'warc.wat.gz')

r = requests.get(warc_url, stream=True)
records = ArchiveIterator(r.raw)

In [10]:
record = next(records)

In [11]:
record.rec_type

'warcinfo'

In [12]:
a = record.content_stream().read()

In [13]:
print(a.decode('utf-8'))

isPartOf: CC-MAIN-2020-24
publisher: Common Crawl
description: Wide crawl of the web for May/June 2020
operator: Common Crawl Admin (info@commoncrawl.org)
hostname: ip-10-67-67-182.ec2.internal
software: Apache Nutch 1.16 (modified, https://github.com/commoncrawl/nutch/)
robots: checked via crawler-commons 1.1-SNAPSHOT (https://github.com/crawler-commons/crawler-commons)
format: WARC File Format 1.1
conformsTo: http://iipc.github.io/warc-specifications/specifications/warc-format/warc-1.1/



In [14]:
r = requests.get(wet_url, stream=True)
records = ArchiveIterator(r.raw)
record = next(records)
record.rec_type
a = record.content_stream().read()
print(a.decode('utf-8'))

Software-Info: ia-web-commons.1.1.10-SNAPSHOT-20200605094634
Extracted-Date: Sun, 07 Jun 2020 16:56:24 GMT
robots: checked via crawler-commons 1.1-SNAPSHOT (https://github.com/crawler-commons/crawler-commons)
isPartOf: CC-MAIN-2020-24
operator: Common Crawl Admin (info@commoncrawl.org)
description: Wide crawl of the web for May/June 2020
publisher: Common Crawl




In [15]:
record = next(records)
record.rec_type

'conversion'

In [16]:
record.rec_headers.headers

[('WARC-Type', 'conversion'),
 ('WARC-Target-URI', 'http://002397.cn/related_report/detail.php?id=866619'),
 ('WARC-Date', '2020-05-25T05:11:44Z'),
 ('WARC-Record-ID', '<urn:uuid:3020cc7c-fd30-4f3d-bbd7-8513f33cd83a>'),
 ('WARC-Refers-To', '<urn:uuid:10bc1a42-8c88-4369-a04e-7b77ca106e79>'),
 ('WARC-Block-Digest', 'sha1:QBZTGL7G53UVVTAZ5VOKXL3C7LRZ2FUR'),
 ('WARC-Identified-Content-Language', 'zho'),
 ('Content-Type', 'text/plain'),
 ('Content-Length', '9300')]

In [17]:
record.http_headers
a = record.content_stream().read()
print(a.decode('utf-8')[:1000])

纺织服装行业周报:终端零售回暖,板块业绩等待验证 - 相关研报 - 梦洁股份(002397)
梦洁股份(002397)
移动版
首页
股票行情
媒体报道
相关新闻
公司公告
研究报告
相关研报
纺织服装行业周报:终端零售回暖,板块业绩等待验证
发布时间：2017-02-12 研究机构：海通证券
投资要点:
市场回顾:本周(20170206-20170212)纺织服装板块上涨2.21%,跑赢上证综指0.41个百分点,在申万一级行业中列第十一。其中,纺织制造板块上涨2.58%,服装家纺板块上涨1.97%。个股方面,万里马、梦洁股份(002397)、摩登大道、美欣达、金发拉比等个股涨幅居前;探路者、星期六、希努尔、比音勒芬、山东如意跌幅靠前。从PE估值水平来看,纺织服装板块目前估值32.9倍(TTM,剔除负值),其中纺织制造板块32.0倍,服装家纺板块35.4倍。
行业数据:零售方面,春节黄金周零售大幅回升,全国百家重点大型零售企业零售额同比增长2.8%,增速相比上年回升了9.4个百分点。其中服装类商品零售额同比增长4.1%,高于上年春节10.1个百分点。2017年1月份,全国50家重点大型零售企业零售额同比增长17.8%,这一增速与同样包含了春节假期的2014年1月份增速基本持平,高于2012年同期增速4.3个百分点,消费市场显示出较强的活力。出口方面,1月份出口现开门红。2017年1月,我国纺织品出口95.84亿美元,同比增长3.50%,服装及其附件出口143.20亿美元,同比增长1.85%,纺织品服装合计出口239.04亿美元,同比增长2.5%。 周组合跑赢行业指数:跨境通(+7.34%),歌力思(+1.13%),美盛文化(+0.18%),乔治白(+2.23%),按照各1/4的权重,组合收益+2.72%。
周观点: 本周,我们对纺织服装板块2016年业绩前瞻进行了整理,共计57家公司先后发布了2016E业绩预告,其中,33家预告业绩增长,5家预告业绩持平/下滑(+5%~-10%),19家预告业绩下滑。 纺织行业率先回暖。我们认为纺织制造子版块业绩有所改善的主要原因有:1)一方面制造业出口比例较高,受益于人民币贬值带来的出口形势的改善,以及部分汇兑损益对报表带来的正面影响;2)制造业下游客户多为优质品牌商,其中海外龙头品牌由于其全球销售的性质,更广泛享受消费复苏的影响

In [18]:
record.rec_headers.get_header('WARC-Target-URI')

'http://002397.cn/related_report/detail.php?id=866619'